# Start Simulation

In [16]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [2]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [3]:
usd_path = f"omniverse://localhost/Users/{user}/uva_sac.usd"

In [4]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-1cd2b635-18ac-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[1.860s] [ext: omni.kit.menu.create-1.0.2] startup
[1.862s] [ext: omni.mdl-0.1.0] startup
[1.884s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.885s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.886s] [ext: omni.kit.material.library-1.3.10] startup
[1.889s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.889s] [ext: omni.kit.window.file-1.3.16] startup
[1.892s] [ext: omni.kit.context_menu-1.3.9] startup
[1.896s] [ext: omni.kit.window.property-1.6.3] startup
[1.897s] [ext: omni.kit.window.content_browser-2.4.28] startup
[1.907s] [ext: omni.kit.widget.stage-2.6.15] startup
[1.911s] [ext: omni.isaac.version-1.0.0] startup
[1.912s] [ext: omni.kit.property.usd-3.14.9] startup
[1.946s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[1.950s] [ext: omni.hydra.rtx-0.1.0] startup
[1.962s] [ext: omni.renderer-rtx-0.0.0] startup
[1.962s] [ext: omn

In [5]:
pause

NameError: name 'pause' is not defined

# Init Env

In [ ]:
from uva_env import UvaEnv
env = UvaEnv()

In [ ]:
from task.utils import add_scene_default
add_scene_default()

In [ ]:
print(list(env.stage.TraverseAll()))

In [ ]:
env.clean()
env.world.step(render=True)

# Config

In [ ]:
from task.config import DATA_PATH
task_type = "Table"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

# Scene

In [ ]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [ ]:
# add base
scene.add_base_asset()

In [ ]:
# add room
# scene.add_room()

In [ ]:
env.world.step(render=True)

# Reward

In [ ]:
from uv.reward import Rewarder

In [ ]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [ ]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras()
render.set_cameras()

In [ ]:
pause

#  Learning

In [ ]:
# from learning.network.resnet import ResNetFeatureExtractor

from learning.utils import extract_image_clip_feature_and_save
from transformers import CLIPProcessor, CLIPModel

feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Trajectory

In [ ]:
total_traj = 0

In [ ]:
# traj config
for traj_id in range(50):
    total_traj += 1
    
    base_asset_id = 0
    env.scene.base_asset_id = base_asset_id
    env.scene.traj_id = traj_id
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))

    # base
    # scene.add_base_asset()
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    
    ## extract feature
    
#     extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#         feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
        f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(5):

        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if total_traj < 10 and np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            

        env.put_last_object((x, y)) 
        env.world.step(render=True)

        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
#         extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#             feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))
        extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
            f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    print("record: ", record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)

# Debug

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

# Replay Buffer

In [6]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(50):
    replay = json.load(open(f"/home/yizhou/Research/DeepArrange/Data/Table/Border/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [7]:
len(buffer.dataset)

250

In [8]:
from learning.network.sac import *

In [9]:
policy = Policy()

In [10]:
qf1 = QFunction()
qf2 = QFunction()
target_qf1 = QFunction()
target_qf2 = QFunction()

In [11]:
from learning.sac_trainer import SACTrainer

In [12]:
trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = False, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

In [13]:
self = trainer

In [18]:
for _ in range(500):
    batch = buffer.sample_batch(batch_size=6)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 20 == 0:
        dist = self.policy(obs, obj_features)
        pred = self.qf1(obs, obj_features, actions)
        print(_, #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
              "\n pred:", pred.flatten().tolist(),
              "\n rewards: ", rewards.flatten().tolist())
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())

0 
 pred: [1.2536238431930542, 1.12417733669281, 1.0520554780960083, 1.1366177797317505, 1.1194133758544922, 0.9137171506881714] 
 rewards:  [-0.43062591552734375, -0.43062591552734375, -0.4306907653808594, -0.43062591552734375, -0.43062591552734375, -0.4306640625]
20 
 pred: [1.1713083982467651, 0.7595911622047424, 1.2111036777496338, 0.8652575016021729, 0.9620705246925354, 0.8531972169876099] 
 rewards:  [-0.43062591552734375, -0.43062591552734375, -0.43062591552734375, -0.43062591552734375, -0.43062591552734375, -0.4306640625]
40 
 pred: [0.7837340831756592, 1.157118558883667, 0.7220757603645325, 0.763007402420044, 0.8227475881576538, 0.8498975038528442] 
 rewards:  [-0.43062591552734375, 0.9153774380683899, -0.43062591552734375, 0.4338764548301697, -0.4306640625, -0.43062591552734375]
60 
 pred: [1.2766077518463135, 0.9344422221183777, 1.0881030559539795, 1.161471962928772, 1.391080617904663, 0.8132531642913818] 
 rewards:  [0.9845327734947205, -0.43062591552734375, -0.4306640625, 

In [19]:
dist = self.policy(obs.to(device), obj_features.to(device))

In [21]:
dist.sample()

tensor([[-0.3332, -0.6917],
        [ 0.7419, -0.1004],
        [ 0.4325,  0.7237],
        [-0.7603,  0.3015],
        [ 0.1949, -0.0783],
        [-0.7992, -0.6562]], device='cuda:0')